List of things that need to be implemented:
- axisymmetric variational formula and L2 error.
- $u_{\theta}=0 $ and $\frac{\partial}{\partial \theta}$=0
- boundary conditions defined well
- Maybe we need a boundary condition on the pressure at r=0/r=1. Would this be dirichlet ?

Variational form for each boundary :

We have on $\partial \Omega $:

$\int_{\partial\Omega} \mu (\nabla u +(\nabla u)^T)\cdot n - pn \, ds$ 

At $r=0$, $\nabla u \cdot n=0$ and $\nabla p \cdot n = 0$. So we have
$\int_{0}^1 \mu (\nabla u)^T\cdot n - pn \, dz$. 

At $r=1$, $u = 0$ and $\nabla p \cdot n = 0$. So we have 0, since $v=0$.

At $z=0$, $\nabla u \cdot n=0$ and $ p= 4$. So we have
$\int_{0}^1 \mu (\nabla u)^T\cdot n - pn \,r\, dr$. 

At $z=1$, $\nabla u \cdot n=0$ and $ p= 0$. So we have
$\int_{0}^1 \mu (\nabla u)^T\cdot n \,r\, dr$. 

In [351]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista

from dolfinx.fem import Constant, Function, functionspace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical, locate_dofs_topological
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter, XDMFFile
from dolfinx.mesh import create_unit_square, exterior_facet_indices, locate_entities_boundary, locate_entities, meshtags, Mesh
from dolfinx.plot import vtk_mesh
from basix.ufl import element
from ufl import (FacetNormal, Identity, TestFunction, TrialFunction, as_matrix,as_vector, SpatialCoordinate, Measure,
                 div, dot, ds, dx, inner, lhs, nabla_grad,grad, rhs, sym)


mesh = create_unit_square(MPI.COMM_WORLD, 10, 10)
t = 0
T = 1
num_steps = 5
dt = T / num_steps

In [352]:
v_cg2 = element("Lagrange", mesh.topology.cell_name(), 2, shape=(mesh.geometry.dim, ))
s_cg1 = element("Lagrange", mesh.topology.cell_name(), 1)
V = functionspace(mesh, v_cg2)
Q = functionspace(mesh, s_cg1)

In [353]:
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

In [354]:
boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], 1)),
              (3, lambda x: np.isclose(x[1], 0)),
              (4, lambda x: np.isclose(x[1], 1))]

facet_indices, facet_markers = [], []
fdim = mesh.topology.dim - 1
for (marker, locator) in boundaries:
    facets = locate_entities(mesh, fdim, locator)
    facet_indices.append(facets)
    facet_markers.append(np.full_like(facets, marker))
facet_indices = np.hstack(facet_indices).astype(np.int32)
facet_markers = np.hstack(facet_markers).astype(np.int32)
sorted_facets = np.argsort(facet_indices)
facet_tag = meshtags(mesh, fdim, facet_indices[sorted_facets], facet_markers[sorted_facets])

mesh.topology.create_connectivity(mesh.topology.dim-1, mesh.topology.dim)
with XDMFFile(mesh.comm, "facet_tags.xdmf", "w") as xdmf:
    xdmf.write_mesh(mesh)
    xdmf.write_meshtags(facet_tag, mesh.geometry)

#ds = Measure("exterior_facet", domain=mesh, subdomain_data=facet_tag)

wall_dofs_2 = locate_dofs_topological(V, fdim, facet_tag.find(1))
u_noslip_2 = np.array([0,1], dtype=PETSc.ScalarType)
bc_noslip_2 = dirichletbc(u_noslip_2, wall_dofs_2, V)

wall_dofs = locate_dofs_topological(V, fdim, facet_tag.find(2))
u_noslip = np.array([0,0], dtype=PETSc.ScalarType)
bc_noslip = dirichletbc(u_noslip, wall_dofs, V)

inflow_dofs = locate_dofs_topological(Q, fdim, facet_tag.find(3))
outflow_dofs = locate_dofs_topological(Q, fdim, facet_tag.find(4))
bc_inflow = dirichletbc(PETSc.ScalarType(4), inflow_dofs, Q)
bc_outflow = dirichletbc(PETSc.ScalarType(0), outflow_dofs, Q)

bcu = [bc_noslip]
bcp = [bc_inflow, bc_outflow]

In [355]:
print(facet_tag.indices)


[  0   3   4   6  12  14  23  25  37  39  54  56  74  76  97  99 123 125
 152 154 183 184 186 211 213 235 237 256 258 274 276 289 291 301 303 310
 312 316 318 319]


In [356]:
'''tdim = mesh.topology.dim
mesh.topology.create_connectivity(tdim - 1, tdim)
boundary_facets = exterior_facet_indices(mesh.topology)

def symmetric(x):
    return np.isclose(x[0], 0)
def wall(x):
    return  np.isclose(x[0], 1)
def inflow(x):
    return np.isclose(x[1], 0)
def outflow(x):
    return np.isclose(x[1], 1)

symmetric_facets = locate_entities_boundary(mesh, tdim - 1, symmetric)
wall_facets = locate_entities_boundary(mesh, tdim - 1, wall)
inflow_facets = locate_entities_boundary(mesh, tdim - 1, inflow)
outflow_facets = locate_entities_boundary(mesh, tdim - 1, outflow)

num_facets = mesh.topology.index_map(tdim - 1).size_local
markers = np.zeros(num_facets, dtype=np.int32)
symmetric_tag = 1
wall_tag = 2
inflow_tag = 3
outflow_tag = 4
markers[symmetric_facets] = symmetric_tag
markers[wall_facets] = wall_tag
markers[inflow_facets] = inflow_tag
markers[outflow_facets] = outflow_tag
facet_marker = meshtags(mesh, tdim - 1, np.arange(num_facets, dtype=np.int32), markers)

#symmetric_dofs = locate_dofs_topological(V, facet_marker.dim, facet_marker.find(symmetric_tag))
#x = SpatialCoordinate(mesh)
#bc_symmetric_p = dirichletbc(np.array([0,lambda x: 4*(1-x[1])], dtype = np.float32 ) , symmetric_dofs, Q)

####################################################################################

# topological BCs

wall_dofs = locate_dofs_topological(V, facet_marker.dim, facet_marker.find(wall_tag))
u_noslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
bc_noslip = dirichletbc(u_noslip, wall_dofs, V)

inflow_dofs = locate_dofs_topological(Q, facet_marker.dim, facet_marker.find(inflow_tag))
outflow_dofs = locate_dofs_topological(Q, facet_marker.dim, facet_marker.find(outflow_tag))
bc_inflow = dirichletbc(PETSc.ScalarType(4), inflow_dofs, Q)
bc_outflow = dirichletbc(PETSc.ScalarType(0), outflow_dofs, Q)


####################################################################################

# Geometric BCs

#wall_dofs = locate_dofs_geometrical(V, wall)
#u_noslip = np.array((0,) * mesh.geometry.dim, dtype=PETSc.ScalarType)
#bc_noslip = dirichletbc(u_noslip, wall_dofs, V)

#inflow_dofs = locate_dofs_geometrical(Q, inflow)
#bc_inflow = dirichletbc(PETSc.ScalarType(4), inflow_dofs, Q)

#outflow_dofs = locate_dofs_geometrical(Q, outflow)
#bc_outflow = dirichletbc(PETSc.ScalarType(0), outflow_dofs, Q)

bcu = [bc_noslip]
bcp = [bc_inflow, bc_outflow]'''

'tdim = mesh.topology.dim\nmesh.topology.create_connectivity(tdim - 1, tdim)\nboundary_facets = exterior_facet_indices(mesh.topology)\n\ndef symmetric(x):\n    return np.isclose(x[0], 0)\ndef wall(x):\n    return  np.isclose(x[0], 1)\ndef inflow(x):\n    return np.isclose(x[1], 0)\ndef outflow(x):\n    return np.isclose(x[1], 1)\n\nsymmetric_facets = locate_entities_boundary(mesh, tdim - 1, symmetric)\nwall_facets = locate_entities_boundary(mesh, tdim - 1, wall)\ninflow_facets = locate_entities_boundary(mesh, tdim - 1, inflow)\noutflow_facets = locate_entities_boundary(mesh, tdim - 1, outflow)\n\nnum_facets = mesh.topology.index_map(tdim - 1).size_local\nmarkers = np.zeros(num_facets, dtype=np.int32)\nsymmetric_tag = 1\nwall_tag = 2\ninflow_tag = 3\noutflow_tag = 4\nmarkers[symmetric_facets] = symmetric_tag\nmarkers[wall_facets] = wall_tag\nmarkers[inflow_facets] = inflow_tag\nmarkers[outflow_facets] = outflow_tag\nfacet_marker = meshtags(mesh, tdim - 1, np.arange(num_facets, dtype=np.

In [357]:
u_n = Function(V)
u_n.name = "u_n"
U = 0.5 * (u_n + u)
n = FacetNormal(mesh)
f = Constant(mesh, PETSc.ScalarType((0, 0)))
k = Constant(mesh, PETSc.ScalarType(dt))
mu = Constant(mesh, PETSc.ScalarType(1))
rho = Constant(mesh, PETSc.ScalarType(1))

In [358]:
# Define strain-rate tensor

def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor

def sigma(u, p):
    return 2 * mu * epsilon(u) - p * Identity(len(u))

# Define the variational problem for the first step
#ds = Measure("ds", domain=mesh, subdomain_data=facet_marker)
x = SpatialCoordinate(mesh)
def div_cyl(u):
    return u[0]+x[0]*(u[0].dx(0)+u[1].dx(1))
def der_dir(u):
    return as_vector([u[0]*u[0].dx(0) + u[1]*u[0].dx(1) , 
                      u[0]*u[1].dx(0) + u[1]*u[1].dx(1)])

p_n = Function(Q)
p_n.name = "p_n"
F1 = rho * dot((u - u_n) / k, v) *x[0]* dx
F1 +=  rho * dot(dot(u_n, nabla_grad(u_n)), v) *x[0]* dx
F1 += inner(sigma(U, p_n), epsilon(v)) *x[0]* dx

F1 += dot(p_n * n, v) *x[0]* ds - dot(mu * nabla_grad(U) * n, v) *x[0]* ds

#F1 += dot(p_n * n, v) *x[0]* ds(1) - dot(mu * nabla_grad(U) * n, v) *x[0]* ds(1)
#F1 += dot(p_n * n, v) *x[0]* ds(2) - dot(mu * nabla_grad(U) * n, v) *x[0]* ds(2)
#F1 += dot(p_n * n, v) *x[0]* ds(3) - dot(mu * nabla_grad(U) * n, v) *x[0]* ds(3)
#F1 += dot(p_n * n, v) *x[0]* ds(4) - dot(mu * nabla_grad(U) * n, v) *x[0]* ds(4)


F1 -= dot(f, v) *x[0]* dx
a1 = form(lhs(F1))
L1 = form(rhs(F1))

In [359]:
A1 = assemble_matrix(a1, bcs=bcu)
A1.assemble()
b1 = create_vector(L1)

In [360]:
# Define variational problem for step 2
u_ = Function(V)
a2 = form(dot(nabla_grad(p), nabla_grad(q)) *x[0]* dx)
L2 = form(dot(nabla_grad(p_n), nabla_grad(q)) *x[0]* dx - (rho / k) * div_cyl(u_) *q* dx)
A2 = assemble_matrix(a2, bcs=bcp)
A2.assemble()
b2 = create_vector(L2)

# Define variational problem for step 3
p_ = Function(Q)
a3 = form(rho * dot(u, v) *x[0]* dx)
L3 = form(rho * dot(u_, v) *x[0]* dx - k * dot(nabla_grad(p_ - p_n), v) *x[0]* dx)
A3 = assemble_matrix(a3)
A3.assemble()
b3 = create_vector(L3)

In [361]:
# Solver for step 1
solver1 = PETSc.KSP().create(mesh.comm)
solver1.setOperators(A1)
solver1.setType(PETSc.KSP.Type.BCGS)
pc1 = solver1.getPC()
pc1.setType(PETSc.PC.Type.HYPRE)
pc1.setHYPREType("boomeramg")

# Solver for step 2
solver2 = PETSc.KSP().create(mesh.comm)
solver2.setOperators(A2)
solver2.setType(PETSc.KSP.Type.BCGS)
pc2 = solver2.getPC()
pc2.setType(PETSc.PC.Type.HYPRE)
pc2.setHYPREType("boomeramg")

# Solver for step 3
solver3 = PETSc.KSP().create(mesh.comm)
solver3.setOperators(A3)
solver3.setType(PETSc.KSP.Type.CG)
pc3 = solver3.getPC()
pc3.setType(PETSc.PC.Type.SOR)

In [362]:
from pathlib import Path

folder = Path("results")
folder.mkdir(exist_ok=True, parents=True)
vtx_u = VTXWriter(mesh.comm, folder / "poiseuille_u.bp", u_n, engine="BP4")
vtx_p = VTXWriter(mesh.comm, folder / "poiseuille_p.bp", p_n, engine="BP4")
vtx_u.write(t)
vtx_p.write(t)

In [363]:
def u_exact(x):
    values = np.zeros((2, x.shape[1]), dtype=PETSc.ScalarType)
    values[1] = 1-x[0]**2
    return values


u_ex = Function(V)
u_ex.interpolate(u_exact)

L2_error = form(dot(u_ - u_ex, u_ - u_ex) *x[0]* dx)

In [364]:
for i in range(num_steps):
    # Update current time step
    t += dt

    # Step 1: Tentative veolcity step
    with b1.localForm() as loc_1:
        loc_1.set(0)
    assemble_vector(b1, L1)
    apply_lifting(b1, [a1], [bcu])
    b1.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b1, bcu)
    solver1.solve(b1, u_.x.petsc_vec)
    u_.x.scatter_forward()

    # Step 2: Pressure corrrection step
    with b2.localForm() as loc_2:
        loc_2.set(0)
    assemble_vector(b2, L2)
    apply_lifting(b2, [a2], [bcp])
    b2.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b2, bcp)
    solver2.solve(b2, p_.x.petsc_vec)
    p_.x.scatter_forward()

    # Step 3: Velocity correction step
    with b3.localForm() as loc_3:
        loc_3.set(0)
    assemble_vector(b3, L3)
    b3.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    solver3.solve(b3, u_.x.petsc_vec)
    u_.x.scatter_forward()
    # Update variable with solution form this time step
    u_n.x.array[:] = u_.x.array[:]
    p_n.x.array[:] = p_.x.array[:]

    # Write solutions to file
    vtx_u.write(t)
    vtx_p.write(t)

    # Compute error at current time-step
    error_L2 = np.sqrt(mesh.comm.allreduce(assemble_scalar(L2_error), op=MPI.SUM))
    error_max = mesh.comm.allreduce(np.max(u_.x.petsc_vec.array - u_ex.x.petsc_vec.array), op=MPI.MAX)
    # Print error only every 20th step and at the last step
    if (i % 20 == 0) or (i == num_steps - 1):
        print(f"Time {t:.2f}, L2-error {error_L2:.2e}, Max error {error_max:.2e}")
# Close xmdf file
vtx_u.close()
vtx_p.close()
b1.destroy()
b2.destroy()
b3.destroy()
solver1.destroy()
solver2.destroy()
solver3.destroy()

Time 0.20, L2-error 2.94e-01, Max error 8.00e-01
Time 1.00, L2-error 2.24e-01, Max error 1.54e+00


In [365]:
topology, cell_types, geometry = vtk_mesh(V)
values = np.zeros((geometry.shape[0], 3), dtype=np.float64)
values[:, :len(u_n)] = u_n.x.array.real.reshape((geometry.shape[0], len(u_n)))

# Create a point cloud of glyphs
function_grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
function_grid["u"] = values
glyphs = function_grid.glyph(orient="u", factor=0.2)

# Create a pyvista-grid for the mesh
mesh.topology.create_connectivity(mesh.topology.dim, mesh.topology.dim)
grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh, mesh.topology.dim))


# Create plotter
plotter = pyvista.Plotter()
plotter.add_mesh(grid, style="wireframe", color="k")


plotter.add_mesh(glyphs)
plotter.view_xy()
if not pyvista.OFF_SCREEN:
    plotter.show()
else:
    fig_as_array = plotter.screenshot("glyphs.png")

Widget(value='<iframe src="http://localhost:44053/index.html?ui=P_0x7f41f63f9f90_39&reconnect=auto" class="pyv…

In [366]:
def plot_mesh(mesh: Mesh, facet_tag = None):
    
    plotter = pyvista.Plotter()
    
    V_linear = functionspace(mesh, ("Lagrange", 1))
    linear_grid = pyvista.UnstructuredGrid(*vtk_mesh(V_linear))
    edges = linear_grid.extract_feature_edges()
    fm = np.zeros(linear_grid.extract_feature_edges().n_cells, dtype=int)
    print(linear_grid.extract_feature_edges().n_cells)
    for index in range(len(facet_tag.values)):
        fm[facet_tag.indices[index]] = facet_tag.values[index]
    print(fm)
    edges['facet_marker'] = fm
    plotter.add_mesh(linear_grid, show_edges=True)
    plotter.add_mesh(edges, scalars='facet_marker', line_width=5, cmap='viridis')

    # We plot the coordinate axis and align it with the xy-plane
    plotter.show_axes()
    plotter.view_xy()
    if not pyvista.OFF_SCREEN:
        plotter.show()

In [367]:
plot_mesh(mesh, facet_tag)


320
[3 0 0 3 2 0 2 0 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0
 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 4
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1 0 0 0 0 0 0 0 0 0 0 0 0 4 0 1 0 0 0 0
 0 0 0 0 0 4 0 1 0 0 0 0 0 0 4 0 1 0 0 0 4 0 1 4]


Widget(value='<iframe src="http://localhost:44053/index.html?ui=P_0x7f41f63f9d10_40&reconnect=auto" class="pyv…